# Préparation de la vérité terrain

vérité terrain= ground truth

## settings

### Monter gdrive

"Monter" un disque dur sur un serveur signifie: les brancher ensemble.

Vous possedez tous un disque dur (virtuel) qui s'appelle google-drive. On va le brancher sur votre instance du serveur colab. Ainsi tout le travail sera sauvarder chez vous.


In [1]:
from google.colab import drive
drive.mount('/gdrive')

In [2]:
"si cela a déjà été fait, cela ne fera rien"
%mkdir /gdrive/My\ Drive/deep_data

In [3]:
%cd /gdrive/My\ Drive/deep_data

### imports

In [4]:
%reset -f



### Dossiers et paramètres globaux

In [5]:
import shutil
import os
import time
import numpy as np
import matplotlib.pyplot as plt
import time
from IPython.display import Image
import time
import json
import scipy
#from sklearn.feature_extraction import image

In [6]:
"le répertoire où l'on va stocker les données"
"changer de nom de répertoire pour ne pas écraser l'ancien"
DIR_FOR_DATA="data_madeToday"

In [7]:
"""ATTENTION, ICI ON DÉTRUIT TOUT CE QUI A DÉJÀ ÉTÉ ENREGISTRÉ"""
if os.path.exists(DIR_FOR_DATA):
    shutil.rmtree(DIR_FOR_DATA)

"""création des dossiers """
os.makedirs(DIR_FOR_DATA)

##  Chargeons et observons les données

Maintenant on transfert les données, de github vers votre gdrive.

In [8]:
if not os.path.exists("data_cells"):
    !git clone https://github.com/vincentvigon/data_cells
else:
    print("images already here")

In [9]:
"data_cells contains two sub-directories with images and ground truth"
%ls data_cells

In [10]:
%ls data_cells/NormalizedPNG/

Here are the image names.

In [11]:
names_str="""TCGA-CH-5767-01Z-00-DX1.png
TCGA-DK-A2I6-01A-01-TS1.png
TCGA-E2-A14V-01Z-00-DX1.png
TCGA-18-5592-01Z-00-DX1.png
TCGA-E2-A1B5-01Z-00-DX1.png
TCGA-21-5784-01Z-00-DX1.png
TCGA-G2-A2EK-01A-02-TSB.png
TCGA-21-5786-01Z-00-DX1.png
TCGA-G9-6336-01Z-00-DX1.png
TCGA-38-6178-01Z-00-DX1.png
TCGA-G9-6348-01Z-00-DX1.png
TCGA-49-4488-01Z-00-DX1.png
TCGA-G9-6356-01Z-00-DX1.png
TCGA-50-5931-01Z-00-DX1.png
TCGA-G9-6362-01Z-00-DX1.png
TCGA-A7-A13E-01Z-00-DX1.png
TCGA-G9-6363-01Z-00-DX1.png
TCGA-A7-A13F-01Z-00-DX1.png
TCGA-HE-7128-01Z-00-DX1.png
TCGA-AR-A1AK-01Z-00-DX1.png
TCGA-HE-7129-01Z-00-DX1.png
TCGA-AR-A1AS-01Z-00-DX1.png
TCGA-HE-7130-01Z-00-DX1.png
TCGA-AY-A8YK-01A-01-TS1.png
TCGA-KB-A93J-01A-01-TS1.png
TCGA-B0-5698-01Z-00-DX1.png
TCGA-NH-A8F7-01A-01-TS1.png
TCGA-B0-5710-01Z-00-DX1.png
TCGA-RD-A8N9-01A-01-TS1.png
TCGA-B0-5711-01Z-00-DX1.png"""

In [12]:
NAMES=np.array(sorted(names_str.split("\n")))
len(NAMES)

Pensez-vous à ce stade qu'il y a suffisament d'image pour faire du deeplearning ?

### Observons les images

In [13]:
img = plt.imread("data_cells/NormalizedPNG/"+NAMES[0])
img.shape,img.dtype

In [14]:
plt.imshow(img);

L'encodage des couleurs en 3 channels, est spécifique à ce type d'image médical, il fait particulièrement resortir les différents types de tissus.

***A vous:*** Quel représente les taches foncées $(1\heartsuit)$?

In [15]:
"les pixels prennent leur valeur entre 0 et 1"
np.min(img),np.max(img)

On va réduire la résolution pour économiser de la place mémoire. Vérifions visuellement si cela ne "déteriore" pas trop l'aspect de l'image.


In [16]:
img_zoomed_order_1=scipy.ndimage.zoom(img,zoom=(0.5,0.5,1),order=1)

In [17]:
plt.imshow(img_zoomed_order_1);

In [18]:
img_zoomed_order2=scipy.ndimage.zoom(img,zoom=(0.5,0.5,1),order=2)

In [19]:
fig,(ax0,ax1,ax2)=plt.subplots(1,3,figsize=(20,10))
ax0.imshow(img[:200,:200,:])
ax0.set_title("original")
ax1.imshow(img_zoomed_order2[:100,:100,:])
ax1.set_title("interpolé ordre 2")
ax2.imshow(img_zoomed_order_1[:100,:100,:])
ax2.set_title("interpolé ordre 1");

On voit que le zoom détériore pas mal les images. Dans l'idéal il faudrait travailler avec les images en entier (mais cela serait plus lourd dans le cadre d'un T.P). Par contre, on n'a pas l'impression que l'interpolation d'ordre 2 change les choses.

In [20]:
tranche=100
fig,(ax0,ax1,ax2)=plt.subplots(1,3,figsize=(20,5))
ax0.plot(img[:200,tranche,0])
ax0.set_title("original")
ax1.plot(img_zoomed_order2[:100,tranche//2,0])
ax1.set_title("interpolé ordre 2")
ax2.plot(img_zoomed_order_1[:100,tranche//2,0])
ax2.set_title("interpolé ordre 1");

***A vous:*** L'interpolation d'ordre 1: c'est l'interpolation bilinéaire: en gros, on calcul des barycentres.

L'interpolation d'ordre 2,3,4... c'est les splin: on crée des polynomes par morceaux de degrés 2,3,4... Cela lisse mieux, l'inconvénient c'est que cela ne respecte pas forcément le fameux "principe du maximum".

Un application $f$ transforman les signaux $s$ en  $f(s)$ respecte le principe du maximum quand
$$
\min s\leq \min f(s)\leq \max f(s) \leq max s
$$

Notre image originale prend ses valeurs dans $[0,1]$ tandis que:

In [21]:
np.min(img_zoomed_order_1),np.max(img_zoomed_order_1)

In [22]:
np.min(img_zoomed_order2),np.max(img_zoomed_order2)

### Observons la vérité terrain (=ground truth)

In [23]:
gt = plt.imread("data_cells/GroundTruthInside/"+NAMES[0])
gt.shape,gt.dtype,np.unique(gt)

In [24]:
img= plt.imread("data_cells/NormalizedPNG/"+NAMES[0])
gt = plt.imread("data_cells/GroundTruthInside/"+NAMES[0])

fig,(ax0,ax1)=plt.subplots(1,2,figsize=(12,12))
ax0.imshow(img)
ax1.imshow(gt)
ax0.axis("off")
ax1.axis("off")
ax0.set_title("X")
ax1.set_title("Y");

Ainsi la vérité terrain: c'est des images binaires, prenant

* La valeur 0 sur l'extérieur des cellules
* La valeur 1 sur l'intérieur des cellules


***A vous:*** D'après vous, comment a été créee cette vérité terrain?

***A vous:*** Avec quelle commande numpy peut-on vérifier que cette image ne prend que 2 valeur $(2\heartsuit)$?

### créons un tenseur d'images

In [25]:
def readFiles(img_indexes,order=1):
    Xs=[]
    Ys=[]

    for name in NAMES[img_indexes]:
        img = plt.imread("data_cells/NormalizedPNG/"+name)
        img=scipy.ndimage.zoom(img,zoom=(0.5,0.5,1),order=order)
        Xs.append(img)


    for name in NAMES[img_indexes]:
        label = plt.imread("data_cells/GroundTruthInside/" + name)
        label=scipy.ndimage.zoom(label,zoom=(0.5,0.5),order=order)
        Ys.append(label)

    Xs=np.array(Xs)
    Ys=np.array(Ys)

    """
     0->outside (see latter on)
     1->tissue
     2->cell
    """
    Ys=Ys+1

    return Xs,Ys[:,:,:,None]

On a prévu 3 catégories.

* 1 -> le tissu entre les cellules
* 2 -> les cellules
* (on se garde 0 pour tout à l'heure)

In [26]:
"on extrait 2 images seulement pour tester les programmes qui suivent"
X_file,Y_file=readFiles([0,1])
print(X_file.shape,Y_file.shape)

In [27]:
np.unique(Y_file)

***Important:*** On voit que l'interposation fait apparaitre des valeurs différentes de 0, 1, 2. Il en sera de même avec les rotations qu'on va introduire plus loin. On fera les arrondis plus loins.

In [28]:
plt.imshow(Y_file[0,:,:,0])
plt.colorbar();

##  Augmentation des données

Souvenez vous, quand les données était des nombres réels, on se permettait de rajouter leur carré, leur cube, leur sinus... Les données sont des images, rajoutons leur rotations.


***A vous:***

* $(1\heartsuit)$ Pour quel type d'image ne faudrait-il pas rajouter les rotations?
* $(1\heartsuit)$ Serait-il malin de rajouter les translations, sachant qu'on va ensuite utliser un réseau convolutif?
* $(1\heartsuit)$ Quelle autre transformation pourrait-on rajouter?


Testons la rotation. Attention, l'angle est en degré.

In [29]:
img_rot=scipy.ndimage.rotate(X_file[1,:,:,:], angle=30, reshape=False,order=1)

In [30]:
fig,(ax0,ax1)=plt.subplots(1,2)
ax0.imshow(X_file[1,:,:,:])
ax1.imshow(img_rot);

Une fonction pour effectuer plusieurs rotations sur X ou sur Y

In [31]:
def dataAugmentation(Z,is_Y):

    padding=int(Z.shape[1]*0.1)

    factor=10

    s=Z.shape

    Z_pad=np.zeros((s[0],s[1]+2*padding,s[2]+2*padding,s[3]))
    Z_pad[:,padding:s[1]+padding,padding:s[2]+padding,:]=Z

    Z_aug=np.zeros((s[0]*factor,s[1]+2*padding,s[2]+2*padding,s[3]))

    angles = np.linspace(5, 360, factor)

    k=0
    for i in range(s[0]):
        for angle in angles:
            """attention, avec order=2, cela crée des dépassements d'entier->pixel rouges"""
            Z_aug[k,:,:,:]=scipy.ndimage.rotate(Z_pad[i,:,:,:], angle=angle, reshape=False,order=1)
            k+=1

    if is_Y:
        res=np.round(Z_aug).astype(np.uint8)
    else:
        res=Z_aug.astype(np.float32)

    return res

Pour le padding, on peut aussi utiliser, `np.pad`

In [32]:
A=np.ones([5,10])
np.pad(A,[(2,2),(3,3)])#,mode="reflect"

***A vous:*** Pourquoi est-ce important de ne pas oublié le `np.round` ?

In [33]:
X_aug=dataAugmentation(X_file,False)
Y_aug=dataAugmentation(Y_file,True)
print(X_aug.shape,Y_aug.shape)

In [34]:
print(np.unique(Y_aug)),Y_aug.dtype

In [35]:
nb=3
fig, axs=plt.subplots(nb,2)

for i in range(nb):
    axs[i,0].imshow(X_aug[i,:,:,:])
    axs[i,1].imshow(Y_aug[i,:,:,0])

***A vous:*** On aurait pu se passer du padding. Mais quel  aurait été l'inconvénient $(1\heartsuit)$?

## Sauvons le tenseur d'images augmentées

In [36]:
def saveOneKindOfData(trainOrTest):

    if trainOrTest=="train":
        X_file,Y_file=readFiles(range(5,30))
    elif trainOrTest=="test":
        X_file,Y_file=readFiles(range(0,5))

    X_aug=dataAugmentation(X_file,False)
    Y_aug=dataAugmentation(Y_file,True)


    print("data preprocessing done")

    "we save data on dist, to restart quicker the next time"
    path_X= f"{DIR_FOR_DATA}/X_{trainOrTest}.npy"
    np.save(path_X,X_aug)
    print(f"X saved at {path_X}")

    path_Y= f"{DIR_FOR_DATA}/Y_{trainOrTest}_unit8.npy"
    np.save(path_Y,Y_aug)

    print(f"Y saved at {path_Y}")

    return X_aug,Y_aug

In [37]:
%%time
X_test,Y_test=saveOneKindOfData("test")

In [38]:
np.unique(Y_test),Y_test.dtype

In [39]:
%%time
X_train,Y_train=saveOneKindOfData("train")

In [40]:
np.unique(Y_train),Y_train.dtype


Nous avons décider de stocker sur le disque toutes les images tournées. L'habitude en deeplearning est plutôt de faire de l'enrichissement "à la volée" c.à.d: on fait l'enrichissement batch après batch avant de les rentrer dans le modèle.  

***A vous:*** $(4\heartsuit)$ Dans quel cas une technique est préférable à une autre?  Plus généralement, indiquez en quelque mot ce qu'est le dilemme "stockage versus calcul" (pas besoin de chercher sur le net, c'est du bon sens).